# We have a dataset from Idealista

## Let's start with data cleaning

### Check the file and info on the document

In [1]:
import pandas as pd

df = pd.read_excel("Houses for rent in Madrid.xlsx")
print(df.head())

   Id       District                                     Address Number  \
0   1  Ciudad Lineal                            Piso en Quintana    NaN   
1   2  Ciudad Lineal               Piso en calle de Arturo Soria    NaN   
2   3  Ciudad Lineal              Piso en calle de Vicente Muzas      4   
3   4  Ciudad Lineal                       Piso en calle Badajoz    NaN   
4   5  Ciudad Lineal  Piso en calle de Nuestra Señora del Villar      9   

          Area  Rent  Bedrooms  Sq.Mt  Floor  Outer  Elevator  Penthouse  \
0     Quintana  1300       2.0     72    3.0    1.0       1.0          0   
1  Costillares  3000       5.0    260    2.0    1.0       1.0          0   
2       Colina  1300       2.0    100    3.0    1.0       1.0          0   
3  San Pascual  1600       3.0    120    4.0    1.0       1.0          0   
4       Ventas   800       2.0     60    3.0    1.0       0.0          0   

   Cottage  Duplex  Semidetached  
0        0       0             0  
1        0       0    

In [2]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2089 entries, 0 to 2088
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            2089 non-null   int64  
 1   District      2089 non-null   object 
 2   Address       2089 non-null   object 
 3   Number        747 non-null    object 
 4   Area          2085 non-null   object 
 5   Rent          2089 non-null   int64  
 6   Bedrooms      2000 non-null   float64
 7   Sq.Mt         2089 non-null   int64  
 8   Floor         1948 non-null   float64
 9   Outer         1927 non-null   float64
 10  Elevator      1956 non-null   float64
 11  Penthouse     2089 non-null   int64  
 12  Cottage       2089 non-null   int64  
 13  Duplex        2089 non-null   int64  
 14  Semidetached  2089 non-null   int64  
dtypes: float64(4), int64(7), object(4)
memory usage: 244.9+ KB
None


In [3]:
nbr_total_rows = df["Id"].count()
dct = {}
for column in df.columns:
    if df[column].isna().sum() != 0:
        NaN_nbr = df[column].isna().sum()
        NaN_perc = NaN_nbr/nbr_total_rows *100
        #print(f"The percentage of NaNs in this column is {column}, and it has a percentage of {NaN_perc}% of NaNs")
        dct[column] = float(NaN_perc)
    #dct = dct.sorted()
print(f"the columns that have NaNs, and their % of NaNs are: {dct}")


the columns that have NaNs, and their % of NaNs are: {'Number': 64.24126376256582, 'Area': 0.19147917663954045, 'Bedrooms': 4.260411680229775, 'Floor': 6.749640976543801, 'Outer': 7.754906653901389, 'Elevator': 6.366682623264721}


#### So the one with the biggest amount of NaNs is clearly "Number"
apart from Number having 64% NaNs, all others have a small number of NaNs

Let's understand why we have NULLS and if we should replace or delete them.

In [4]:
del df["Number"]
print(df.info)

<bound method DataFrame.info of         Id       District                                     Address  \
0        1  Ciudad Lineal                            Piso en Quintana   
1        2  Ciudad Lineal               Piso en calle de Arturo Soria   
2        3  Ciudad Lineal              Piso en calle de Vicente Muzas   
3        4  Ciudad Lineal                       Piso en calle Badajoz   
4        5  Ciudad Lineal  Piso en calle de Nuestra Señora del Villar   
...    ...            ...                                         ...   
2084  2184      Vicálvaro                   Piso en calle Villajoyosa   
2085  2185      Vicálvaro                         Piso en Los Ángeles   
2086  2186      Vicálvaro                          Piso en San Andrés   
2087  2187      Vicálvaro   Piso en calle del Doctor MartÃ­n ArÃ©valo   
2088  2188      Vicálvaro            Estudio en avenida Real de Pinto   

             Area  Rent  Bedrooms  Sq.Mt  Floor  Outer  Elevator  Penthouse  \
0        Qui

Bedrooms if NaN => there's only 1 big room with everything

Floor if NaN => it's probably a house

Outer ==> No clue, so maybe leave out...

Could try to impute for the floor outer and elevators if they're NULL and hav the name "Casa or Chalet"

Maybe drop the NULL Areas.


Do it and we'll say that we did it to the group

In [ ]:
mask =df["Bedrooms"].isna() 
mask2 = df["Address"].str.contains(r"\bEstudio\b") 
print(df[mask2]["Address"].count())#92 
print(df[mask][mask2]["Address"].count())#82 
mask3 = df["Bedrooms"].notna() 
print(df[mask3][mask2]["Bedrooms"].mean())#0.0 so they're all 0, so that's what I'll replace the other Null Bedrooms that have Estudios
df.loc[mask2 & mask, "Bedrooms"] = 0.0 
#print(df[mask2 & mask]["Bedrooms"]) #It worked 

92
82
0.0


/var/folders/dz/mp6v9x4n2y30h186jvh_7lh40000gn/T/ipykernel_22505/863629351.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[mask][mask2]["Address"].count())#82
/var/folders/dz/mp6v9x4n2y30h186jvh_7lh40000gn/T/ipykernel_22505/863629351.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[mask3][mask2]["Bedrooms"].mean())#0.0 so they're all 0, so that's what I'll replace the other Null Bedrooms that have Estudios


In [ ]:
#print(df[mask2]["Address"].isna().sum())#0
#print(df[mask]["Address"])
mask2 = df["Address"].str.contains(r"\bÁtico\b")
print(df[mask2]["Bedrooms"].isna().sum())#5 instances 
print(df[mask2]["Bedrooms"].mean())#So no real mean, but only 5 of them are Null, so we can drop them
print(df["Bedrooms"].isna().sum())#Only 2 aren't accounted for yet, so we can drop them...
df = df.dropna(subset=["Bedrooms"])
print(df["Bedrooms"].isna().sum())


5
2.3353658536585367
7
0


## Bedrooms is done
Now let's work on Floors

In [ ]:
mask = ~df["Floor"].isna() 
#mask2 = ~(df["Address"].str.contains(r"\bPiso\b") | df["Address"].str.contains(r"\bChalet\b") | df["Address"].str.contains(r"\bchalet\b"))
mask2 = (df["Address"].str.contains(r"\bChalet\b") | df["Address"].str.contains(r"\bchalet\b")) 
print(df[mask][mask2]["Address"])#Apart from Chalet & Piso, only 5 others that are Caserón and 4 Estudio
print(df[mask][mask2]["Address"].count())#141, 83 are Chalets, 53 are Pisos => 136 in total, what are the other 5?
mask3 = df["Floor"].notna() 
print(df[mask][mask2]["Floor"].mean())#8607.8.... Maybe we should look at outliers too, not just NaNs...
mask_outliers = df["Floor"] < 30 
print(df[mask_outliers]["Floor"].mean())#3.56, so there's definitely an outliers
mask_find_outliers = df["Floor"] > 30
print(df[mask_find_outliers])#3.56, so there's definitely an outliers, there's one with 43k floors.... (and it's a cottage without an elevator)

1418                        Chalet adosado en El Viso
1805                          Chalet adosado en Rejas
1808              Chalet adosado en calle Maria Tarin
1814    Casa o chalet independiente en calle de Teseo
1923       Chalet adosado en calle de la vÁ­a lÁ­mite
Name: Address, dtype: object
5
8607.8
3.5618556701030926
        Id   District                    Address        Area  Rent  Bedrooms  \
1418  1485  Chamartín  Chalet adosado en El Viso  en El Viso  7500       5.0   

      Sq.Mt    Floor  Outer  Elevator  Penthouse  Cottage  Duplex  \
1418    310  43039.0    NaN       NaN          0        1       0   

      Semidetached  
1418             1  


C:\Users\Romain\AppData\Local\Temp\ipykernel_20016\3547332991.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[mask][mask2]["Address"])#Apart from Chalet & Piso, only 5 others that are Caserón and 4 Estudio
C:\Users\Romain\AppData\Local\Temp\ipykernel_20016\3547332991.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[mask][mask2]["Address"].count())#141, 83 are Chalets, 53 are Pisos => 136 in total, what are the other 5?
C:\Users\Romain\AppData\Local\Temp\ipykernel_20016\3547332991.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[mask][mask2]["Floor"].mean())#8607.8.... Maybe we should look at outliers too, not just NaNs...


Now let's do 'Outer', but we have to understand what it is if we want to know if we should drop them or apply something else...

In [ ]:
mask = df["Outer"].isna() 
mask2 = ~(df["Address"].str.contains(r"\bPiso\b") | df["Address"].str.contains(r"\bChalet\b") | df["Address"].str.contains(r"\bchalet\b"))
print(df[mask][mask2]["Address"])# 
print(df[mask][mask2]["Address"].count())#8 are not Chalet or Piso (1 Caserón, 5 Ãticos, and 2 Estudios)

112               Caserón en calle Río Bullaque
487     Estudio en calle de Martín de los Heros
675                               Ático en Goya
749      Ático en calle del príncipe de vergara
1220                           Ático en Palacio
1303                 Estudio en Chueca-Justicia
1621                           Ático en Almagro
1672               Ático en calle donoso cortés
Name: Address, dtype: object
8


C:\Users\Romain\AppData\Local\Temp\ipykernel_20016\2377254352.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[mask][mask2]["Address"])#
C:\Users\Romain\AppData\Local\Temp\ipykernel_20016\2377254352.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[mask][mask2]["Address"].count())#8 are not Chalet or Piso (1 Caserón, 5 Ãticos, and 2 Estudios)


Now let's check Elevators:

In [124]:
mask = df["Elevator"].isna() 
mask2 = ~(df["Address"].str.contains(r"\bPiso\b") | df["Address"].str.contains(r"\bChalet\b") | df["Address"].str.contains(r"\bchalet\b"))
print(df[mask][mask2]["Address"])#Apart from Chalet & Piso, only 5 others that are Caserón and 4 Estudio
print(df[mask][mask2]["Address"].count())#133 total,  3 not Chalet or Piso (Caserón, a Estudio and an Ático)
mask3 = df["Elevator"].notna()
print(df["Elevator"].mean())#3.56, so there's definitely an outliers
mask_find_outliers = df["Elevator"] > 1
print(df[mask_find_outliers])#None, so no Outlier here

112              Caserón en calle Río Bullaque
487    Estudio en calle de Martín de los Heros
675                              Ático en Goya
Name: Address, dtype: object
3
0.8804515135967163
Empty DataFrame
Columns: [Id, District, Address, Area, Rent, Bedrooms, Sq.Mt, Floor, Outer, Elevator, Penthouse, Cottage, Duplex, Semidetached]
Index: []


C:\Users\Romain\AppData\Local\Temp\ipykernel_20016\2798802313.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[mask][mask2]["Address"])#Apart from Chalet & Piso, only 5 others that are Caserón and 4 Estudio
C:\Users\Romain\AppData\Local\Temp\ipykernel_20016\2798802313.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[mask][mask2]["Address"].count())#133 total,  3 not Chalet or Piso (Caserón, a Estudio and an Ático)
